#This notebook will be used for the IBM Data Science Specialization Capstone Project.

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium

print("Libraries imported.")

Libraries imported.


In [3]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [6]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [7]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [11]:
toronto_df.shape

(180, 3)

In [8]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


In [10]:
toronto_df_dropna.shape

(103, 3)

In [12]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [13]:
# print the number of rows
toronto_df_grouped.shape

(103, 3)

In [14]:
csv_path = 'https://cocl.us/Geospatial_data'
df = pd.read_csv(csv_path)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df.shape

(103, 3)

In [16]:
df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
toronto_df_grouped.iloc[0,0]

'M1B\n'

In [19]:
df.iloc[0,0] 

'M1B'

In [21]:
1 == 1

True

In [22]:
toronto_df_grouped.iloc[0,0] == df.iloc[0,0]  #Can you see, that they are not the same that's the reason they are not able to merge as they are supposed to

False

In [36]:
toronto_df_grouped['Neighborhood'] = toronto_df_grouped['Neighborhood'].str.replace(' /', ',')

In [37]:
toronto_df_grouped['PostalCode'] = toronto_df_grouped['PostalCode'].str.replace('\n', '')

In [38]:
toronto_df_grouped['Borough'] = toronto_df_grouped['Borough'].str.replace('\n', '')

In [40]:
toronto_df_grouped.head() #See, now the data is good to go

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
#we can also check the equality condition
toronto_df_grouped.iloc[0,0] == df.iloc[0,0] 

True

In [43]:
#Now we can merge it easily
toronto_df_new2 = pd.merge(left=df, right=toronto_df_grouped, left_on='PostalCode', right_on='PostalCode')
toronto_df_new2.head() 
#There you go

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [70]:
toronto_df_new = toronto_df_grouped.merge(df, on="PostalCode", how='left')
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,Malvern / Rouge,NaN,NaN
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,NaN,NaN
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill,NaN,NaN
3,M1G\n,Scarborough\n,Woburn,NaN,NaN
4,M1H\n,Scarborough\n,Cedarbrae,NaN,NaN


In [73]:
# merge two table on the column "PostalCode"
toronto_df_new2 = pd.merge(left=df, right=toronto_df_grouped, left_on='PostalCode', right_on='PostalCode')
toronto_df_new2

,PostalCode,Latitude,Longitude,Borough,Neighborhood


In [53]:
result = pd.concat([toronto_df_grouped, df], axis = 1, sort=False)
result

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B\n,Scarborough\n,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G\n,Scarborough\n,Woburn,M1G,43.770992,-79.216917
4,M1H\n,Scarborough\n,Cedarbrae,M1H,43.773136,-79.239476
5,M1J\n,Scarborough\n,Scarborough Village,M1J,43.744734,-79.239476
6,M1K\n,Scarborough\n,Kennedy Park / Ionview / East Birchmount Park,M1K,43.727929,-79.262029
7,M1L\n,Scarborough\n,Golden Mile / Clairlea / Oakridge,M1L,43.711112,-79.284577
8,M1M\n,Scarborough\n,Cliffside / Cliffcrest / Scarborough Village West,M1M,43.716316,-79.239476
9,M1N\n,Scarborough\n,Birch Cliff / Cliffside West,M1N,43.692657,-79.264848


In [74]:
toronto_df_new3 = toronto_df_grouped.merge(left='df', right='toronto_df_grouped')
toronto_df_new3.head()

TypeError: merge() got an unexpected keyword argument 'left'

In [58]:
df2=pd.read_csv('C:\Users\Owner\Documents\Coursera courses\Data Science Specialization\Course 9 Capstone Project\Geospatial_Coordinates.csv')
print (df2)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-58-f96b1b808eac>, line 1)